In [16]:
# 1  numpy和pytorch实现梯度下降，我们用一个简单的函数来验证一下
import numpy as np
# N是批次的大小，D_in为输入维度，H为隐藏层维度，D_out为输出层维度
N,D_in,H,D_out = 64,1000,100,5
# 使用numpt产生x
np.random.seed(2019)
x = np.random.randn(N, D_in)
y = np.random.randn(N, D_out)
# y1 = w1*x + b ,我们将偏置一起产生
w1 = np.random.randn(D_in, H)
# y2 = w2*y1 + b 
w2 = np.random.randn(H,D_out)

# 设定梯度下降的学习率
learning_rate = 1e-6
for i in range(280):
    # 前向传播：计算y值
    h = x.dot(w1)
    # 使用relu函数进行非线性变换
    h_relu = np.maximum(h, 0)
    # 输出最后一层结果
    y_pred = h_relu.dot(w2)
    
    # 计算loss
    loss = np.square(y_pred - y).sum()
    print("第{}次loss为{}".format(i, loss))
    
    # 反向传播计算梯度,可以自己手推一下
    grad_y_pred = 2.0 * (y_pred - y)
    grad_w2 = h_relu.T.dot(grad_y_pred)
    grad_h_relu = grad_y_pred.dot(w2.T)
    grad_h = grad_h_relu.copy()
    grad_h[h < 0] = 0
    grad_w1 = x.T.dot(grad_h)
    
    # 更新权重
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2  

第0次loss为10181441.287914116
第1次loss为7287253.004034052
第2次loss为6476611.662577391
第3次loss为7222047.772760225
第4次loss为9591840.38883196
第5次loss为13823323.253201079
第6次loss为19915951.28301371
第7次loss为26644101.40939445
第8次loss为31053783.469333615
第9次loss为29406812.98017192
第10次loss为21639077.880555607
第11次loss为12320947.672297576
第12次loss为5800105.95501852
第13次loss为2492355.8313350654
第14次loss为1081700.17963597
第15次loss为515977.2990621543
第16次loss为288388.29376166756
第17次loss为191247.27705143654
第18次loss为144782.1399632446
第19次loss为118709.41577614087
第20次loss为101467.63537437504
第21次loss为88566.21949285068
第22次loss为78132.04946338062
第23次loss为69333.63927567485
第24次loss为61763.691346997875
第25次loss为55170.704060452
第26次loss为49390.85974253587
第27次loss为44300.23857060188
第28次loss为39801.04048122349
第29次loss为35813.88986002903
第30次loss为32273.246218184115
第31次loss为29125.9294212675
第32次loss为26317.154388893927
第33次loss为23805.023209712454
第34次loss为21556.041220771913
第35次loss为19539.420333888345
第36次loss为17728.728799795637


In [27]:
# 现在我们使用pytorch搭建一个网络
import torch
# 看一下是否可以使用GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# N批大小，D_in输入维度
# H隐藏层维度，D_out输出维度
N,D_in,H,D_out = 64,1000,100,10

# 产生随机数据
x = torch.randn(N, D_in, device=device)
y = torch.randn(N, D_out, device=device)
import torch.nn as nn
# 我们使用nn包快捷的搭建神经网络，可以自动生成权重并使用to方法移动到所需要的设备

model = nn.Sequential(
    nn.Linear(D_in,H),
    nn.ReLU(),
    nn.Linear(H,D_out),
).to(device)

# 同时我们使用nn库搭建常用的损失函数，设置reduction='sum'，表示求的平方误差的和
loss = nn.MSELoss(reduction='elementwise_mean')
learning_rate = 1e-2 # 学习率对于梯度计算影响非常大
for i in range(500):
    # 前向传播，重载_call_运算符 
    y_pred= model(x)
    #计算损失
    loss_data = loss(y_pred, y)
    print("第{}次损失为{}".format(i,loss_data))
    
    # 反向传播之前清0梯度
    model.zero_grad()
    
    # 反向传播，计算模型损失对所有可学习参数的导数，
    loss_data.backward()
    
    # 使用梯度下降更新权重
    with torch.no_grad():
        for param in model.parameters():
            param.data -= learning_rate*param.grad

D:\Anaconda3\lib\site-packages\torch\nn\_reduction.py:16: UserWarning: reduction='elementwise_mean' is deprecated, please use reduction='mean' instead.
  warnings.warn("reduction='elementwise_mean' is deprecated, please use reduction='mean' instead.")


第0次损失为1.1122791767120361
第1次损失为1.0979230403900146
第2次损失为1.0839163064956665
第3次损失为1.0702338218688965
第4次损失为1.0568898916244507
第5次损失为1.0438387393951416
第6次损失为1.031114101409912
第7次损失为1.018725037574768
第8次损失为1.0066238641738892
第9次损失为0.9948293566703796
第10次损失为0.983319103717804
第11次损失为0.9721183776855469
第12次损失为0.9611766338348389
第13次损失为0.9504898190498352
第14次损失为0.940003514289856
第15次损失为0.9297230839729309
第16次损失为0.9196334481239319
第17次损失为0.9097288250923157
第18次损失为0.9000158905982971
第19次损失为0.8904574513435364
第20次损失为0.8810781836509705
第21次损失为0.8718481063842773
第22次损失为0.8627815246582031
第23次损失为0.8539249300956726
第24次损失为0.8452216386795044
第25次损失为0.8366761207580566
第26次损失为0.8283010721206665
第27次损失为0.8200500011444092
第28次损失为0.8119148015975952
第29次损失为0.8039052486419678
第30次损失为0.7960113883018494
第31次损失为0.7882334589958191
第32次损失为0.7805706262588501
第33次损失为0.7730339765548706
第34次损失为0.7656034231185913
第35次损失为0.7582990527153015
第36次损失为0.751110851764679
第37次损失为0.7440206408500671
第38次损失为0.7370378971099854
第

第365次损失为0.03867300972342491
第366次损失为0.03834039717912674
第367次损失为0.038010742515325546
第368次损失为0.03768424317240715
第369次损失为0.03736096993088722
第370次损失为0.03704074025154114
第371次损失为0.036723267287015915
第372次损失为0.036408908665180206
第373次损失为0.036097537726163864
第374次损失为0.03578909859061241
第375次损失为0.03548363223671913
第376次损失为0.03518108278512955
第377次损失为0.034881316125392914
第378次损失为0.0345844142138958
第379次损失为0.034290019422769547
第380次损失为0.03399850055575371
第381次损失为0.03370988741517067
第382次损失为0.03342372179031372
第383次损失为0.03314025327563286
第384次损失为0.032859403640031815
第385次损失为0.032581351697444916
第386次损失为0.032305892556905746
第387次损失为0.03203282132744789
第388次损失为0.03176209330558777
第389次损失为0.03149411082267761
第390次损失为0.031228644773364067
第391次损失为0.030965115875005722
第392次损失为0.030704587697982788
第393次损失为0.03044649213552475
第394次损失为0.03019065223634243
第395次损失为0.029937200248241425
第396次损失为0.029686136171221733
第397次损失为0.029437245801091194
第398次损失为0.029190802946686745
第399次损失为0.02894666977226734
第400次

In [30]:
# 由于手动设置学习率可能比较麻烦，所以我们可以采用一种自适应的方法
learning_rate = 1e-2
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for i in range(500):
    # 前向传播
    y_pred = model(x)
    # 计算Loss
    loss_data = loss(y_pred,y)
    print("第{}次的loss为{:.6f}".format(i,loss_data))
    
    # 梯度清0
    optimizer.zero_grad()
    # 反向传播
    loss_data.backward()
    # 调用adam的step方法更新参数
    optimizer.step()
    

D:\Anaconda3\lib\site-packages\torch\nn\_reduction.py:16: UserWarning: reduction='elementwise_mean' is deprecated, please use reduction='mean' instead.
  warnings.warn("reduction='elementwise_mean' is deprecated, please use reduction='mean' instead.")


第0次的loss为0.000000
第1次的loss为0.756431
第2次的loss为0.128820
第3次的loss为0.384093
第4次的loss为0.277746
第5次的loss为0.132828
第6次的loss为0.078831
第7次的loss为0.081366
第8次的loss为0.090615
第9次的loss为0.083169
第10次的loss为0.062604
第11次的loss为0.043020
第12次的loss为0.034635
第13次的loss为0.035447
第14次的loss为0.035735
第15次的loss为0.030901
第16次的loss为0.023758
第17次的loss为0.019266
第18次的loss为0.018074
第19次的loss为0.017854
第20次的loss为0.016343
第21次的loss为0.013928
第22次的loss为0.011737
第23次的loss为0.010209
第24次的loss为0.009215
第25次的loss为0.008343
第26次的loss为0.007627
第27次的loss为0.007202
第28次的loss为0.006567
第29次的loss为0.005657
第30次的loss为0.004905
第31次的loss为0.004491
第32次的loss为0.004229
第33次的loss为0.003847
第34次的loss为0.003349
第35次的loss为0.002937
第36次的loss为0.002686
第37次的loss为0.002507
第38次的loss为0.002316
第39次的loss为0.002087
第40次的loss为0.001796
第41次的loss为0.001507
第42次的loss为0.001319
第43次的loss为0.001256
第44次的loss为0.001231
第45次的loss为0.001108
第46次的loss为0.000930
第47次的loss为0.000849
第48次的loss为0.000844
第49次的loss为0.000795
第50次的loss为0.000687
第51次的loss为0.000584
第52次的loss为0.000518
第53

第416次的loss为0.000042
第417次的loss为0.000023
第418次的loss为0.000016
第419次的loss为0.000029
第420次的loss为0.000024
第421次的loss为0.000009
第422次的loss为0.000012
第423次的loss为0.000019
第424次的loss为0.000014
第425次的loss为0.000011
第426次的loss为0.000010
第427次的loss为0.000011
第428次的loss为0.000011
第429次的loss为0.000006
第430次的loss为0.000007
第431次的loss为0.000007
第432次的loss为0.000004
第433次的loss为0.000005
第434次的loss为0.000006
第435次的loss为0.000005
第436次的loss为0.000003
第437次的loss为0.000004
第438次的loss为0.000004
第439次的loss为0.000004
第440次的loss为0.000002
第441次的loss为0.000002
第442次的loss为0.000003
第443次的loss为0.000002
第444次的loss为0.000001
第445次的loss为0.000001
第446次的loss为0.000001
第447次的loss为0.000001
第448次的loss为0.000001
第449次的loss为0.000001
第450次的loss为0.000001
第451次的loss为0.000001
第452次的loss为0.000001
第453次的loss为0.000001
第454次的loss为0.000001
第455次的loss为0.000001
第456次的loss为0.000001
第457次的loss为0.000001
第458次的loss为0.000002
第459次的loss为0.000002
第460次的loss为0.000003
第461次的loss为0.000004
第462次的loss为0.000007
第463次的loss为0.000010
第464次的loss为0.000016
第465次的loss为0.000025


In [43]:
# 如果我们想自定义函数可以继承mm.Moudle定义forward函数

class TwoLayerNet(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        """
        在构造函数中，我们实例化了两个nn.Linear模块，并将它们作为成员变量。
        """
        super(TwoLayerNet, self).__init__()
        self.linear1 = torch.nn.Linear(D_in, H)
        self.linear2 = torch.nn.Linear(H, D_out)

    def forward(self, x):
        """
        在前向传播的函数中，我们接收一个输入的张量，也必须返回一个输出张量。
        我们可以使用构造函数中定义的模块以及张量上的任意的（可微分的）操作。
        """
        h_relu = self.linear1(x).clamp(min=0)
        y_pred = self.linear2(h_relu)
        return y_pred

N, D_in, H, D_out = 64, 1000, 100, 10
# 产生输入和输出的随机张量
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

# 我们实例化上面类构建我们的模型
model_1 = TwoLayerNet(D_in,H,D_out)

# 构造优化器和损失函数
loss_fn = nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model_1.parameters(),lr=1e-4)

for i in range(500):
    # 前向传播
    y_pred = model_1(x)
    
    # 计算loss
    loss = loss_fn(y_pred,y)
    print("第{}次的loss为{:.6f}".format(i,loss))
    
    # 清0梯度
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

第0次的loss为638.004272
第1次的loss为589.936584
第2次的loss为547.919434
第3次的loss为511.086700
第4次的loss为478.196198
第5次的loss为448.490845
第6次的loss为421.492554
第7次的loss为396.903656
第8次的loss为374.189026
第9次的loss为353.103210
第10次的loss为333.309784
第11次的loss为314.877869
第12次的loss为297.513550
第13次的loss为281.106476
第14次的loss为265.774536
第15次的loss为251.166183
第16次的loss为237.324066
第17次的loss为224.183990
第18次的loss为211.633133
第19次的loss为199.646774
第20次的loss为188.144394
第21次的loss为177.151672
第22次的loss为166.682693
第23次的loss为156.735123
第24次的loss为147.311737
第25次的loss为138.376526
第26次的loss为129.945038
第27次的loss为121.977287
第28次的loss为114.437500
第29次的loss为107.303925
第30次的loss为100.550873
第31次的loss为94.167664
第32次的loss为88.161552
第33次的loss为82.501839
第34次的loss为77.168541
第35次的loss为72.148712
第36次的loss为67.440872
第37次的loss为63.014889
第38次的loss为58.863308
第39次的loss为54.975624
第40次的loss为51.334007
第41次的loss为47.927380
第42次的loss为44.741947
第43次的loss为41.763748
第44次的loss为38.982132
第45次的loss为36.385250
第46次的loss为33.964676
第47次的loss为31.711452
第48次的loss为29.608477